In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, callbacks
import time, json, os, shutil, argparse
import warnings
warnings.simplefilter("ignore")

In [2]:
epochs = 100
batch_size = 160
labels = ["0", "1"]
dir_models = "./Models/HC/G"
dir_training = "./Data/HC/G/"
dir_validation = "./Data/HC/O/Validation"
img_width, img_height, img_depth = 320, 240, 3
path_model = os.path.join(dir_models, 'resnet.h5')
path_history = os.path.join(dir_models, 'resnet.csv')

In [3]:
resnet_50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
resnet_50.trainable=False

inputs = tf.keras.Input(shape=(img_width, img_height, img_depth))
x = resnet_50(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.Activation('sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name="ResNet50_TL")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Nadam(learning_rate=1e-4, epsilon=1e-08),
              metrics=['accuracy'])
#model.summary()

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    dir_training,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

validation_generator = test_datagen.flow_from_directory(
    dir_validation,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

Found 512 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [5]:
callback_log = callbacks.CSVLogger(path_history, append=True)
callback_model_save = callbacks.ModelCheckpoint(filepath=path_model, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)

In [6]:
time_in = time.time()

hist = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[callback_log, callback_model_save])

time_out = time.time()
print('\n', 'Time cost:', '\n', time_out-time_in)

Epoch 1/100
4/4 [==============================] - 71s 16s/step - loss: 3.6042 - accuracy: 0.4961 - val_loss: 0.8908 - val_accuracy: 0.2000
Epoch 2/100
4/4 [==============================] - 68s 16s/step - loss: 0.5405 - accuracy: 0.7227 - val_loss: 0.9323 - val_accuracy: 0.3500
Epoch 3/100
4/4 [==============================] - 62s 19s/step - loss: 0.8490 - accuracy: 0.6016 - val_loss: 1.1884 - val_accuracy: 0.5000
Epoch 4/100
4/4 [==============================] - 62s 15s/step - loss: 0.6169 - accuracy: 0.7949 - val_loss: 0.9677 - val_accuracy: 0.4500
Epoch 5/100
4/4 [==============================] - 79s 19s/step - loss: 0.3744 - accuracy: 0.8242 - val_loss: 0.9694 - val_accuracy: 0.2000
Epoch 6/100
4/4 [==============================] - 71s 16s/step - loss: 0.2163 - accuracy: 0.9648 - val_loss: 1.2724 - val_accuracy: 0.4500
Epoch 7/100
4/4 [==============================] - 72s 18s/step - loss: 0.5470 - accuracy: 0.7090 - val_loss: 1.0809 - val_accuracy: 0.2000
Epoch 8/100
4/4 [===